In [ ]:
 import six
import numpy as np
import tensorflow.compat.v2 as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

!pip install git+https://github.com/google/qkeras

  Cloning https://github.com/google/qkeras to /tmp/pip-req-build-7a81044c
  Running command git clone -q https://github.com/google/qkeras /tmp/pip-req-build-7a81044c
     |████████████████████████████████| 174kB 12.4MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 6.8MB 15.2MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
  Created wheel for QKeras: filename=QKeras-0.8.0-cp36-none-any.whl size=143338 sha256=40a2ac3891a1652696557c0cb560dbf83fcc74f64d2bcf1adad7904708e8c1de
  Stored in directory: /tmp/pip-ephem-wheel-cache-vw44z8mu/wheels/b4/74/1d/9456d62789716894a5edd7e342b4beaef69241ac584706c68d
  Created wheel for pyparser: filename=pyparser-1.0-cp36-none-any.whl size=4944 sha256=b1b8972692f208a5c1245eb4f3b6b106f34d9688d7d114773fb98aa8708b08b7
  Stored in directory: /root/.cache/pip/wheels/8a/1c/4f/9f66cd69719aa41c2684efae758d95db7803e9fe1f65146db1
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.wh

In [ ]:
from qkeras import *
import qkeras
def CreateQModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)

    x = Flatten(name="flatten")(x)
    x = QDense(256,
        kernel_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(9, {} , alpha = 1)".format(intBits),
        name="dense")(x)
    x = QActivation("quantized_tanh(9, {})".format(intBits), name="act_1")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
        name="dense2")(x)
    x = QActivation("quantized_tanh(9, {})".format(intBits), name="act_2")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
        name="dense3")(x)
    x = QActivation("quantized_tanh(9, {})".format(intBits), name="act_3")(x)
    x = QDense(nb_classes,
        kernel_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
        name="dense4")(x)
    x = Activation("softmax", name="softmax")(x)


    model = Model(inputs=x_in, outputs=x)
    
    return model

In [ ]:
def get_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape + (1,)).astype("float32")
    x_test = x_test.reshape(x_test.shape + (1,)).astype("float32")

    x_train /= 256.0
    x_test /= 256.0

    x_mean = np.mean(x_train, axis=0)

    x_train -= x_mean
    x_test -= x_mean

    nb_classes = np.max(y_train)+1
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = get_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
from qkeras.utils import *

In [ ]:
from tensorflow.keras.optimizers import Adam
histories = []
qmodels = []
for i in range(0, 9):
  qmodel = CreateQModel(x_train.shape[1:], y_train.shape[-1], i)
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)
  qmodel.compile(
      loss="categorical_crossentropy",
      optimizer=Adam(0.0005),
      metrics=["accuracy"])
  history = qmodel.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test[:5000], y_test[:5000]), verbose=False,callbacks=[callback])
  model_save_quantized_weights(qmodel)

  qmodels.append(qmodel)
  histories.append(history)
  print ("Done with model with num of integer bits = {}".format(i))


... quantizing model
Done with model with num of integer bits = 0
... quantizing model
Done with model with num of integer bits = 1
... quantizing model
Done with model with num of integer bits = 2
... quantizing model
Done with model with num of integer bits = 3
... quantizing model
Done with model with num of integer bits = 4
... quantizing model
Done with model with num of integer bits = 5
... quantizing model
Done with model with num of integer bits = 6
... quantizing model
Done with model with num of integer bits = 7
... quantizing model
Done with model with num of integer bits = 8


In [ ]:
for i in range(9):
  qmodels[i].evaluate(x=x_test[5000:], y=y_test[5000:])

157/157 [==============================] - 0s 3ms/step - loss: 3867.5432 - accuracy: 0.6878


# Now for 12 bits

In [ ]:
from qkeras import *
import qkeras
def CreateQModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)

    x = Flatten(name="flatten")(x)
    x = QDense(256,
        kernel_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(13, {} , alpha = 1)".format(intBits),
        name="dense")(x)
    x = QActivation("quantized_tanh(13, {})".format(intBits), name="act_1")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        name="dense2")(x)
    x = QActivation("quantized_tanh(13, {})".format(intBits), name="act_2")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        name="dense3")(x)
    x = QActivation("quantized_tanh(13, {})".format(intBits), name="act_3")(x)
    x = QDense(nb_classes,
        kernel_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        name="dense4")(x)
    x = Activation("softmax", name="softmax")(x)


    model = Model(inputs=x_in, outputs=x)
    
    return model

In [ ]:
from qkeras.utils import *
from tensorflow.keras.optimizers import Adam
histories2 = []
qmodels2 = []
for i in range(0, 13):
  qmodel = CreateQModel(x_train.shape[1:], y_train.shape[-1], i)
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)
  qmodel.compile(
      loss="categorical_crossentropy",
      optimizer=Adam(0.0005),
      metrics=["accuracy"])
  history = qmodel.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test[:5000], y_test[:5000]), verbose=False,callbacks=[callback])
  model_save_quantized_weights(qmodel)

  qmodels2.append(qmodel)
  histories2.append(history)
  print ("Done with model with num of integer bits = {}".format(i))


... quantizing model
Done with model with num of integer bits = 0
... quantizing model
Done with model with num of integer bits = 1
... quantizing model
Done with model with num of integer bits = 2
... quantizing model
Done with model with num of integer bits = 3
... quantizing model
Done with model with num of integer bits = 4
... quantizing model
Done with model with num of integer bits = 5
... quantizing model
Done with model with num of integer bits = 6
... quantizing model
Done with model with num of integer bits = 7
... quantizing model
Done with model with num of integer bits = 8
... quantizing model
Done with model with num of integer bits = 9
... quantizing model
Done with model with num of integer bits = 10
... quantizing model
Done with model with num of integer bits = 11
... quantizing model
Done with model with num of integer bits = 12


In [ ]:
for i in range(13):
  qmodels2[i].evaluate(x=x_test[5000:], y=y_test[5000:])

157/157 [==============================] - 0s 2ms/step - loss: 806845.2500 - accuracy: 0.3200


# Now for 16 bits

In [ ]:
from qkeras import *
import qkeras
def CreateQModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)

    x = Flatten(name="flatten")(x)
    x = QDense(256,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
        name="dense")(x)
    x = QActivation("quantized_tanh(17, {})".format(intBits), name="act_1")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense2")(x)
    x = QActivation("quantized_tanh(17, {})".format(intBits), name="act_2")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense3")(x)
    x = QActivation("quantized_tanh(17, {})".format(intBits), name="act_3")(x)
    x = QDense(nb_classes,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense4")(x)
    x = Activation("softmax", name="softmax")(x)


    model = Model(inputs=x_in, outputs=x)
    
    return model

In [ ]:
from qkeras.utils import *
from tensorflow.keras.optimizers import Adam
histories3 = []
qmodels3 = []
for i in range(0, 17):
  qmodel = CreateQModel(x_train.shape[1:], y_train.shape[-1], i)
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)
  qmodel.compile(
      loss="categorical_crossentropy",
      optimizer=Adam(0.0005),
      metrics=["accuracy"])
  history = qmodel.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test[:5000], y_test[:5000]), verbose=False,callbacks=[callback])
  model_save_quantized_weights(qmodel)

  qmodels3.append(qmodel)
  histories3.append(history)
  print ("Done with model with num of integer bits = {}".format(i))


... quantizing model
Done with model with num of integer bits = 0
... quantizing model
Done with model with num of integer bits = 1
... quantizing model
Done with model with num of integer bits = 2
... quantizing model
Done with model with num of integer bits = 3
... quantizing model
Done with model with num of integer bits = 4
... quantizing model
Done with model with num of integer bits = 5
... quantizing model
Done with model with num of integer bits = 6
... quantizing model
Done with model with num of integer bits = 7
... quantizing model
Done with model with num of integer bits = 8
... quantizing model
Done with model with num of integer bits = 9
... quantizing model
Done with model with num of integer bits = 10
... quantizing model
Done with model with num of integer bits = 11
... quantizing model
Done with model with num of integer bits = 12
... quantizing model
Done with model with num of integer bits = 13
... quantizing model
Done with model with num of integer bits = 14
... q

In [ ]:
for i in range(17):
  qmodels3[i].evaluate(x=x_test[5000:], y=y_test[5000:])

157/157 [==============================] - 0s 2ms/step - loss: 91481448.0000 - accuracy: 0.1190
